# Today we will be playing with CLIP peaks.
- there are two kinds of peaks:

1. IDR peaks: reproducible peaks from two reps. files are here `/home/elvannostrand/data/clip/CLIPseq_analysis/ENCODE_FINALforpapers_20180205/hg38/IDR/UID.01v02.IDR.out.0102merged.bed.blacklist_removed.bed.narrowPeak.bed` (replace `UID` with a specific number)
2. individual replicate peaks are here: `home/elvannostrand/data/clip/CLIPseq_analysis/ENCODE_FINALforpapers_20180205/hg38/` in the format of `UID_02.basedon_204_02.peaks.l2inputnormnew.bed.compressed.bed.blacklist_removed.bed` where `02` means rep2

optional: read about [IDR](https://hbctraining.github.io/Intro-to-ChIPseq/lessons/07_handling-replicates-idr.html)

# You will learn
- How to use BedTool to deal with peaks and gene annotations
- Human Gene Annotations
- UCSC Genome Browser

In [1]:
from pybedtools import BedTool


# BedTool 
- is a package for .bed files and .gff files
- it was a commandline tool. pybedtools is a wrapper for the tool
- Can perform arithematic operation on bedfiles.
- take a look at the [pybedtool tutorial](https://daler.github.io/pybedtools/tutorial-contents.html)
- take a look at several operationss [bedtool intersect](https://bedtools.readthedocs.io/en/latest/content/tools/intersect.html), [bedtools merge](https://bedtools.readthedocs.io/en/latest/content/tools/merge.html), [subtract](https://bedtools.readthedocs.io/en/latest/content/tools/subtract.html)

In [2]:
# let's take a look at the RBFOX2's peak (UID=204)
rbfox_idr = BedTool('/home/elvannostrand/data/clip/CLIPseq_analysis/ENCODE_FINALforpapers_20180205/hg38/IDR/204.01v02.IDR.out.0102merged.bed.blacklist_removed.bed.narrowPeak.bed')

In [3]:
print(rbfox_idr[0])

chr21	45528055	45528135	RBFOX2_HepG2_IDR	1000	-	3.69979283065069	7.88501625625323	-1	-1



In [4]:
rbfox_idr[0].chrom # try .start .end

'chr21'

## IDR file comes in the NarrowPeak format
[read this](https://genome.ucsc.edu/FAQ/FAQformat.html#format12).

each columns are `chrom`, `start`, `end`, `name`, `score`, `strand`,
column 7 is `log2(fold enrichment)` and column 8 is `-log10(pvalue)`.

### Question: what does log2FC=3.69 mean?

### Question: why do we need both p-value and fold change?

### Let's take look at this peak at genome browser
go to [encode project portal](https://www.encodeproject.org/experiments/ENCSR987FTF/), click on file details, there's a "visualize" button on the right upper corner. paste `chr21	45528055	45528135` into the top search box.

1. what gene is it in?
2. is it in intron or exon?

# Let's look at what genes do this RBP bind
but the bed file does not show what genes that peak is in!

## So we need  genome annoatations!
Genome annotations are here in the [gencode website](https://www.gencodegenes.org/human/)

since we are using genome coordinate hg38, make sure not to download v19! (that is for hg19)

## My annotation is here
- unparsed version: `/home/hsher/gencode_coords/gencode.v33.annotation.gff3`
- if you look at the file, you will find there's no feature as "intron"! 
- how would you create the "intron" annotation? (hint: `bedtool subtract`)

## parsing annotation is a headache :(
let's just use what I have already parsed! this file contains intron(written as transcript, exon, utrs)`gencode.v33.combine.sorted.gff3`; for transcripts `gencode.v33.transcript.gff3`

If you are interested in how those parsed files come from, the script is here `/home/hsher/ClipNet/scripts/gencode_canon_filtering.sh`

In [5]:
# I will use my annotations here, the full and unparsed version!
transcript_coords = BedTool('/home/hsher/gencode_coords/gencode.v33.transcript.gff3')

In [6]:
# to findout which transcript we bedtool intersect transcript with peaks
transcripts_bound = transcript_coords.intersect(rbfox_idr, s= True, u = True).saveas() # why do I use option s and u?

In [7]:
len(transcripts_bound)

2337

In [8]:
print(transcripts_bound[0])

chr1	HAVANA	transcript	827598	859446	.	+	.	ID=ENST00000445118.7;Parent=ENSG00000228794.10;gene_id=ENSG00000228794.10;transcript_id=ENST00000445118.7;gene_type=lncRNA;gene_name=LINC01128;transcript_type=lncRNA;transcript_name=LINC01128-205;level=2;transcript_support_level=1;hgnc_id=HGNC:49377;tag=basic;havana_gene=OTTHUMG00000196004.1;havana_transcript=OTTHUMT00000007015.4



In [9]:
# let's look at the type of transcripts they are
from collections import Counter
Counter([t.attrs['transcript_type'] for t in transcripts_bound])

Counter({'lncRNA': 122,
         'protein_coding': 2126,
         'processed_transcript': 15,
         'miRNA': 2,
         'unprocessed_pseudogene': 3,
         'nonsense_mediated_decay': 47,
         'transcribed_unprocessed_pseudogene': 5,
         'TEC': 6,
         'retained_intron': 8,
         'pseudogene': 1,
         'snoRNA': 2})

# You might want to know what functions these transcripts do

common ways of doing this:
- Gene Ontology (GO) Enrichment analysis
- Gene Set enrichment analysis (GSEA)
- Reactome/pathway enrichment analysis

you can google around while I also read more closely about its theory and maybe we can go through next time :). These tools are popular today and a lot of people use them without knowing what they are doing. But I think it is crucial to know the fundementals so that you know its limitations!

# We can also look at what regions these peaks are in!

In [10]:
region_anno = BedTool('/home/hsher/gencode_coords/gencode.v33.combine.sorted.gff3')

In [11]:
print(region_anno[0])

chr1	HAVANA	exon	11869	12227	.	+	.	ID=exon:ENST00000456328.2:1;Parent=ENST00000456328.2;gene_id=ENSG00000223972.5;transcript_id=ENST00000456328.2;gene_type=transcribed_unprocessed_pseudogene;gene_name=DDX11L1;transcript_type=processed_transcript;transcript_name=DDX11L1-202;exon_number=1;exon_id=ENSE00002234944.1;level=2;transcript_support_level=1;hgnc_id=HGNC:37102;tag=basic;havana_gene=OTTHUMG00000000961.2;havana_transcript=OTTHUMT00000362751.1



In [12]:
regions_bound = region_anno.intersect(rbfox_idr, s = True, u = True).saveas()

In [13]:
Counter([r[2] for r in regions_bound]) # transcripts are intron

Counter({'transcript': 3262,
         'exon': 441,
         'CDS': 171,
         'five_prime_UTR': 50,
         'three_prime_UTR': 178})

## knowing which regions in a gene a RBP binds is crucial for it function.

read [fig2 in this paper](https://www.nature.com/articles/s41586-020-2077-3). This is how fig2 is generated! You can write a paper now!

# Pointing you to all existing eCLIP files.

see this [Onboarding document](https://www.notion.so/Yeo-Lab-Wiki-5f1c4c524c9a413b92154b4cc3a5f585?p=bf29f3763797410c9ad813b10cc34289).

Right now there are two phases of ENCORE, ENCORE3 and ENCORE4. ENCORE3 has about 150 RBPs in both cell lines and has already ended. ENCORE 4 is ongoing.

You will need the "menifest file", this files will tell you which `UID` is which protein, and what cell line is it CLIPped.

In [14]:
import pandas as pd

encode3=pd.read_excel('/projects/ps-yeolab5/encode/encode_manifest_final.xlsx')

In [15]:
encode3.columns

Index(['Hiseq_file_name', 'ENCODE_ID', 'RBP', 'inline_1', 'inline_2',
       'index_1', 'index_2', 'Lane', 'file_location', 'unmerged_location',
       'original_file_name', 'is_encode', 'cell_type', 'hiseq_run_date',
       'randomer_length', 'Unnamed: 15', 'Method_Paper_flag', 'species',
       'is_4000', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24'],
      dtype='object')

In [16]:
# but I have a even cleaner version of menifest :) from Eric!
encode3 = pd.read_csv('/home/hsher/projects/RBP_annot/ENCODE_FINAL_ANNOTATIONS.uidsonly.txt.manifesthg38.txt', sep = '\t', header= 0)

In [17]:
encode3.head()

,uID,RBP,Cell line,CLIP_rep1,CLIP_rep2,INPUT
0,203,HNRNPC,HepG2,/projects/ps-yeolab3/encode/analysis/encode_GR...,/projects/ps-yeolab3/encode/analysis/encode_GR...,/projects/ps-yeolab3/encode/analysis/encode_GR...
1,204,RBFOX2,HepG2,/projects/ps-yeolab3/encode/analysis/encode_GR...,/projects/ps-yeolab3/encode/analysis/encode_GR...,/projects/ps-yeolab3/encode/analysis/encode_GR...
2,205,IGF2BP1,HepG2,/projects/ps-yeolab3/encode/analysis/encode_GR...,/projects/ps-yeolab3/encode/analysis/encode_GR...,/projects/ps-yeolab3/encode/analysis/encode_GR...
3,206,HNRNPK,HepG2,/projects/ps-yeolab3/encode/analysis/encode_GR...,/projects/ps-yeolab3/encode/analysis/encode_GR...,/projects/ps-yeolab3/encode/analysis/encode_GR...
4,209,SRSF7,HepG2,/projects/ps-yeolab3/encode/analysis/encode_GR...,/projects/ps-yeolab3/encode/analysis/encode_GR...,/projects/ps-yeolab3/encode/analysis/encode_GR...


In [18]:
# encode 4 files are all over the place, so you might want this
df = pd.read_csv('/home/hsher/projects/ClipNet/archishma/ENCODE4_1214.csv')

In [19]:
df.head() # idr column points you to the directory

,Unnamed: 0,uid,Batch,RBP,prefix,bam_0,bam_1,bam_control,plus_0,plus_1,plus_control,minus_0,minus_1,minus_control,bed_0,bed_1,idr,Cell Line
0,0,4001,batch2,NONO,encode4_batch2.4001,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,K562
1,1,4002,batch2,FXR2,encode4_batch2.4002,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,K562
2,2,4004,batch2,DEK,encode4_batch2.4004,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,K562
3,3,4007,batch2,EIF2B3,encode4_batch2.4007,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,K562
4,4,4008,batch2,RBFOX3,encode4_batch2.4008,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,/projects/ps-yeolab5/encore/processing/encore_...,K562


# What you can consider doing
- look at the list of RBP you found to be important in HepG2 survival. What regions do they bind? What transcripts do they bind?
- Do they regulate other important genes? (you might need to generate a list of "important genes" and use "BedTool filter" to get their coordinates.


** below are advanced questions we can walk through next time **
- Are cancer-dependent genes "overrepresented" in RBP's target? How to establish statistical significance? (hint: Fisher exact test/chi-square tests)
- If you test on multiple RBPs, you might need to correct the p-values a bit. Why? (hint: what does p-value mean?) google "Multiple Hypothesis testing"